<a href="https://colab.research.google.com/github/ruih12/ec601-team/blob/main/preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Data Augmentation Implementation
We can apply brightness, contrast, and noise adjustments as data augmentation techniques during the image preprocessing step. These modifications help enhance the model’s robustness, making it more adaptable to different lighting conditions and noise levels.

Data Augmentation Implementation Code:

In [ ]:
# preprocess.py (Data Augmentation Part)

import os
import cv2
import numpy as np
import random

def adjust_brightness_contrast(image, brightness=0, contrast=0):
    # Adjust the brightness and contrast of an image
    # brightness: [-100, 100] -> negative darkens, positive brightens
    # contrast: [-100, 100] -> negative decreases contrast, positive increases
    img = np.int16(image)
    img = img * (contrast / 127 + 1) - contrast + brightness
    img = np.clip(img, 0, 255)
    return np.uint8(img)

def add_noise(image, noise_level=0.02):
    # Adds Gaussian noise to the image
    # noise_level: controls noise intensity, larger values increase noise
    noise = np.random.normal(0, 255 * noise_level, image.shape).astype(np.uint8)
    noisy_image = cv2.add(image, noise)
    return np.clip(noisy_image, 0, 255)

def augment_image(image):
    # Apply random data augmentation techniques
    # Brightness and contrast adjustment
    brightness = random.randint(-50, 50)
    contrast = random.randint(-50, 50)
    image = adjust_brightness_contrast(image, brightness, contrast)

    # Add noise with a 50% chance
    if random.random() < 0.5:
        image = add_noise(image)

    return image

def preprocess_images_with_augmentation(input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for img_name in os.listdir(input_dir):
        img_path = os.path.join(input_dir, img_name)
        image = cv2.imread(img_path)
        if image is None:
            continue

        # Apply data augmentation
        image = augment_image(image)

        # Save the augmented image
        output_path = os.path.join(output_dir, img_name)
        cv2.imwrite(output_path, image)

if __name__ == '__main__':
    input_directory = 'dataset/historical_figures/'
    output_directory = 'dataset/augmented_figures/'
    preprocess_images_with_augmentation(input_directory, output_directory)


# Explanation:

adjust_brightness_contrast: Adjusts pixel values to control brightness and contrast.

add_noise: Adds Gaussian noise to simulate different image qualities.

augment_image: Randomly applies brightness/contrast adjustments and noise addition.

preprocess_images_with_augmentation: Processes the entire dataset and saves the augmented images.

# 2. Face Mask Implementation
A face mask can be used to better align facial features and avoid focusing on the background. Here, we use Dlib to extract facial landmarks and create a mask over the face region, effectively masking out the non-facial background.

Face Mask Implementation Code:

In [ ]:
# preprocess.py (Face Mask Part)

import dlib
import numpy as np
import cv2

# Initialize face detector and facial landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

def create_face_mask(image, landmarks):
    # Create a face mask based on facial landmarks
    mask = np.zeros_like(image, dtype=np.uint8)

    # Define face contour region
    points = []
    for i in range(0, 17):  # Jawline
        points.append((landmarks.part(i).x, landmarks.part(i).y))
    for i in range(26, 16, -1):  # Connect the top of the eyebrows to the jawline
        points.append((landmarks.part(i).x, landmarks.part(i).y))

    points = np.array(points, dtype=np.int32)

    # Fill the contour area to form the mask
    cv2.fillPoly(mask, [points], (255, 255, 255))

    # Apply mask to the image
    face_only_image = cv2.bitwise_and(image, mask)
    return face_only_image

def preprocess_images_with_mask(input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for img_name in os.listdir(input_dir):
        img_path = os.path.join(input_dir, img_name)
        image = cv2.imread(img_path)
        if image is None:
            continue

        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        faces = detector(gray)

        for face in faces:
            # Detect facial landmarks
            landmarks = predictor(gray, face)
            masked_image = create_face_mask(image, landmarks)

            # Save the masked image
            output_path = os.path.join(output_dir, img_name)
            cv2.imwrite(output_path, masked_image)
            break  # Only process the first detected face

if __name__ == '__main__':
    input_directory = 'dataset/historical_figures/'
    output_directory = 'dataset/masked_figures/'
    preprocess_images_with_mask(input_directory, output_directory)


# Explanation:

create_face_mask: Generates a face contour mask using landmarks. The mask is created by filling a polygon covering the face region, effectively excluding the background.

preprocess_images_with_mask: Applies the face mask to each image in the dataset and saves the result.

# Combining Data Augmentation and Face Mask
We can integrate both data augmentation and face masking into a single preprocessing workflow to further improve image quality and alignment.

Combined Code Example:

In [ ]:
def preprocess_images_with_augmentation_and_mask(input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for img_name in os.listdir(input_dir):
        img_path = os.path.join(input_dir, img_name)
        image = cv2.imread(img_path)
        if image is None:
            continue

        # Apply data augmentation
        image = augment_image(image)

        # Convert to grayscale and detect face
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        faces = detector(gray)

        for face in faces:
            landmarks = predictor(gray, face)
            # Apply face mask
            masked_image = create_face_mask(image, landmarks)

            # Save the processed image
            output_path = os.path.join(output_dir, img_name)
            cv2.imwrite(output_path, masked_image)
            break  # Only process the first detected face

if __name__ == '__main__':
    input_directory = 'dataset/historical_figures/'
    output_directory = 'dataset/augmented_masked_figures/'
    preprocess_images_with_augmentation_and_mask(input_directory, output_directory)


# Combined Workflow:

Data Augmentation: First, the image undergoes brightness/contrast adjustments and noise augmentation to improve robustness under different lighting and noise conditions.

Face Masking: A face mask is applied to the augmented image to isolate the face and exclude any distracting background elements.

Save Processed Image: The final processed image is saved to the specified directory.